# More Users

Ok, the main point of this chapter is to go from our state of assuming that all users will share a common to-do list on the site to the state where we support multiple independent users.  The injunction with which we begin is, as always in this book, to consider first how we would confirm that whatever implementation we choose to realize this goal *is* in fact successful.

Here's an interesting point: he suggests that our functional tests file represents the closest thing that we have for a design document for this project.  I like that way of looking at the TDD paradigm: rather than keeping everything scattered and nebulous in your mind, you could be using the files storing your functional tests as a way to plan around the project; what should go where.

In [2]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import time
import unittest

MAX_WAIT = 10

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def wait_for_row_in_list_table(self, row_text):
        start_time = time.time()
        while True:
            try:
                table = self.browser.find_element_by_id('id_list_table')
                rows = table.find_elements_by_tag_name('tr')
                self.assertIn(row_text, [row.text for row in rows])
                return
            except (AssertionError, WebDriverException) as e:
                if time.time() - start_time > MAX_WAIT:
                    raise e
                time.sleep(0.5)

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)

        # The page updates again, and now shows both items on her list
        self.wait_for_row_in_list_table('1: Buy peacock feathers')
        self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Edith wonders whether the site will remember her list.  Then she sees
        # that the site has generated a unique URL for her -- there is some 
        # explanatory text to that effect.
        self.fail('Finish the test!')

        # She visits that URL - her to-do list is still there.

        # Satisfied, she goes back to sleep

Overwriting functional_tests/tests.py


# Agile vs. "Big Design Up Front"

Ok, another interesting point about design philosophy follows immediately.  Harry says that the traditional way to tackle large engineering projects would be to try to plan everything out with paper (or at least schematic software), figuring out where everything should go before you start writing code.  TDD, on the other hand, was derived from or otherwise aligned with the [AGILE design movement](https://en.wikipedia.org/wiki/Agile_software_development), which emphasizes incremental, iterative design, rather than figuring you can make an overarching plan from the very get-go that will prove very robust to complications encountered when coding up the details.  Basically, get your hands dirty early on, and expect the need to refactor.

Of course, the advocated answer is still not really to just dive in mindlessly, rather, it's to think in mid-sized, modular chunks.  The emphasis is on delivering a "minimal viable product", getting it in the field where you can collect real-world feedback about its reception.  Furthermore, this mindset is meant to counter the temptation to add on additional features and functionality that you might envision in a "perfect" or otherwise optimal app.  The initialism is "**YAGNI**", for "you ain't gonna need it": let the users tell you if something is lacking.

In this case, the plan to support multiple users involves having the server return customized URLs for each user, so they see different content based on the recognition of their particular credentials, and thus previously stored list items.


## REST

There's another term here, the REST (Representational State Transfer) API paradigm, which sounds like its gist is that the URL returned for any part of a web app/site should reflect the "data structure" of the content.  In practice, Harry suggests that the trailing part of the URL for a to-do list should contain:

* `lists/new` for a page that accepts POST requests to generate new lists
* `lists/list_name` to view the contents of an existing list
* `lists/list_name/add_item` for a page that accepts POST requests to add to a list

## Next Steps

He suggests that refactoring to support multiple users will involve creating an entirely new functional test; not sure yet if that means a new file, a new class in our existing module, or just new methods within the existing class.  ...Ok, it is quickly revealed that it's the latter option: new methods tacked on to our existing setup.  Ah, ok, but also some refactoring of existing ones; we're not just going to leave in place the soon-to-be-irrelevant methods that assumed that all visitors to the site shared one list.  So some shuffling around of code within the tests.

In [4]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import time
import unittest

MAX_WAIT = 10

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def wait_for_row_in_list_table(self, row_text):
        start_time = time.time()
        while True:
            try:
                table = self.browser.find_element_by_id('id_list_table')
                rows = table.find_elements_by_tag_name('tr')
                self.assertIn(row_text, [row.text for row in rows])
                return
            except (AssertionError, WebDriverException) as e:
                if time.time() - start_time > MAX_WAIT:
                    raise e
                time.sleep(0.5)

    def test_can_start_a_list_for_one_user(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)

        # The page updates again, and now shows both items on her list
        self.wait_for_row_in_list_table('1: Buy peacock feathers')
        self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Satisfied, she goes back to sleep

    def test_multiple_users_can_start_lists_at_different_urls(self):
        # Edith starts a new to-do list
        self.browser.get(self.live_server_url)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Buy peacock feathers')
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # She notices that her list has a unique URL
        edith_list_url = self.browser.current_url
        self.assertRegex(edith_list_url, '/lists/.+')

        # Now a new user, Francis, comes along to the site.

        ## We use a new browser session to make sure that no information
        ## of Edith's is coming through from cookies, etc.
        self.browser.quit()
        self.browser = webdriver.Firefox()

        # Francis visits the home page.  There is no sign of Edith's list
        self.browser.get(self.live_server_url)
        page_text = self.browser.find_element_by_tag_name('body').text
        self.assertNotIn('Buy peacock feathers', page_text)
        self.assertNotIn('make a fly', page_text)

        # Francis starts a new list by entering a new item.
        # He is less intersting than Edith...
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Buy milk')
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy milk')

        # Francis gets his own unique URL
        francis_list_url = self.browser.current_url
        self.assertRegex(francis_list_url, '/lists.+')
        self.assertNotEqual(francis_list_url, edith_list_url)

        # Again, there is no trace of Edith's list
        page_text = self.browser.find_element_by_tag_name('body').text
        self.assertNotIn('Buy peacock feathers', page_text)
        self.assertIn('Buy milk', page_text)

        # Satisfied, they both go back to sleep

Overwriting functional_tests/tests.py


Harry explains that the double-hash-mark lines represent *meta-comments*, which is really a bit of an aside to the dev reviewing the test, explaining *why* a particular step is being coded into the test that the user themselves wouldn't necessarily execute.  It's breaking the flow of assuming that the comments and code all represent a user's experience in navigating the site.  Here, it's assumed that another dev wouldn't understand why we're closing the browser (because it's not yet clear that a separate user is about to come on).

Now we run the modified functional test, hoping that `test_can_start_a_list_for_one_user` passes and `test_multiple_users_can_start_lists_at_different_urls` will fail...

In [1]:
!python manage.py test functional_tests

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.F
FAIL: test_multiple_users_can_start_lists_at_different_urls (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 80, in test_multiple_users_can_start_lists_at_different_urls
    self.assertRegex(edith_list_url, '/lists/.+')
AssertionError: Regex didn't match: '/lists/.+' not found in 'http://localhost:64258/'

----------------------------------------------------------------------
Ran 2 tests in 15.636s

FAILED (failures=1)


And that *is* the outcome anticipated by the text.  He suggests a commit to the remote repo at this point.


# Increments... again

We rehash the main objectives summarized for addressing this chapter:

1. Adjust the "model" of the website so that items are associated with unique lists
2. Make unique URLs for each list
3. A unique URL for generating new lists
4. A unique URL for *adding* items to extant lists

And Harry says that the effect of the error encountered above is indicating that the second item isn't working yet.  Specifically, we aren't being redirected as expected after a POST request is issued.  We need to change the location expected when redirected in the `test_redirects_after_post` method of the **unit tests**, and since we haven't succeeded yet in breaking out the model into unique lists for every user, he suggests a placeholder value that's obviously inadequate to the final purpose: "`the-only-list-in-the-world`".

In [3]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(
            response['location'], '/lists/the-only-list-in-the-world'
        )

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)

    def test_displays_all_list_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/')

        self.assertIn('itemey 1', response.content.decode())
        self.assertIn('itemey 2', response.content.decode())


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


Overwriting lists/tests.py


So in this case, we modified line #21 of the unit tests file.  And, apparently, we now anticipate this test will fail:

In [4]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


....F.
FAIL: test_redirects_after_POST (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 21, in test_redirects_after_POST
    response['location'], '/lists/the-only-list-in-the-world'
AssertionError: '/' != '/lists/the-only-list-in-the-world'
- /
+ /lists/the-only-list-in-the-world


----------------------------------------------------------------------
Ran 6 tests in 0.038s

FAILED (failures=1)


Having confirmed that the unit test does indeed yield the expected failure, he says we can proceed to modifying the dev script, specifically the `views` file which determines which URL to return after POSTing.

In [7]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/lists/the-only-list-in-the-world')

    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})


Overwriting lists/views.py


Check that the unit test now passes:

In [8]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


......
----------------------------------------------------------------------
Ran 6 tests in 0.033s

OK


But the functional tests break, because the page doesn't exist:

In [2]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


......EE
ERROR: test_can_start_a_list_for_one_user (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 55, in test_can_start_a_list_for_one_user
    self.wait_for_row_in_list_table('1: Buy peacock feathers')
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 28, in wait_for_row_in_list_table
    raise e
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 22, in wait_for_row_in_list_table
    table = self.browser.find_element_by_id('id_list_table')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 360, in find_element_by_id
    return self.find_element(by=By.ID, value=id_)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", l

He notes that the fact that an older test, which was previously passing, now fails, is known as a *regression*.  Kind of perversely, he suggests ameliorating the situation by actually *making* a page known as `the-only-list-in-the-world`.  Sounds ridiculous to me, but I guess the supposed advantage is that having the infrastructure in place to constitute a fix will ultimately be expanded and repurposed to serve as the *actual* content of the site in the future, when we're ready to get around to that.

The proposed solution in this case involves creating a new class in the `lists/tests.py` file:

In [5]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(
            response['location'], '/lists/the-only-list-in-the-world'
        )

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)

    def test_displays_all_list_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/')

        self.assertIn('itemey 1', response.content.decode())
        self.assertIn('itemey 2', response.content.decode())


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


class ListViewTest(TestCase):

    def test_displays_all_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/lists/the-only-list-in-the-world/')

        self.assertContains(response, 'itemey 1')
        self.assertContains(response, 'itemey 2')

Overwriting lists/tests.py


He points out that one new change here is getting away from the more verbose `assertIn` and `response.content.decode` approaches, and instead utilizing the `assertContains` method contained within Django, "*which knows how to deal with responses and the bytes of their content*".

Retry the test:

In [6]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


......FEE
ERROR: test_can_start_a_list_for_one_user (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 55, in test_can_start_a_list_for_one_user
    self.wait_for_row_in_list_table('1: Buy peacock feathers')
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 28, in wait_for_row_in_list_table
    raise e
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 22, in wait_for_row_in_list_table
    table = self.browser.find_element_by_id('id_list_table')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 360, in find_element_by_id
    return self.find_element(by=By.ID, value=id_)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", 

He says that the `assertContains` helps you pick out more detail from the resulting tests, that it can tell that the response code is 404, which is just because we haven't implemented the URL.


## A New URL

We implement this by editing the `superlists/urls.py` file:

In [77]:
%%writefile superlists/urls.py
from django.conf.urls import url
from lists import views

urlpatterns = [
    url(r'^$', views.home_page, name='home'),
    url(
        r'^lists/the-only-list-in-the-world/$', views.view_list, name='view_list'
    ),
]


Overwriting superlists/urls.py


What's new there is the eighth line (noting that because I don't remember having checked out that file's contents before).

Rerun the test:

In [78]:
%%capture func_test
!python manage.py test

In [83]:
lines = func_test.stderr.split('\n')
# print('\n'.join(lines[-5:]))

print('\n'.join(lines[:len(lines) // 3]))

......EEE
ERROR: test_displays_all_items (lists.tests.ListViewTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 64, in test_displays_all_items
    response = self.client.get('/lists/the-only-list-in-the-world/')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 536, in get
    **extra)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 340, in get
    return self.generic('GET', path, secure=secure, **r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 416, in generic
    return self.request(**r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 501, in request
    six.reraise(*exc_info)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-p

## New View Function

Kind of unexpectedly, Harry recommends addressing this issue by just adding an empty function to the `views` file:

In [84]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/lists/the-only-list-in-the-world')

    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})

def view_list(request):
    pass

Overwriting lists/views.py


In [90]:
%%capture func_test_1
!python manage.py test

In [91]:
print(func_test.stderr[-500:])

ver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: Unable to locate element: [id="id_list_table"]


----------------------------------------------------------------------
Ran 9 tests in 34.128s

FAILED (errors=3)



In [92]:
# print(func_test_1.stderr[-500:])
print(func_test_1.stderr)

......EEE
ERROR: test_displays_all_items (lists.tests.ListViewTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 64, in test_displays_all_items
    response = self.client.get('/lists/the-only-list-in-the-world/')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 536, in get
    **extra)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 340, in get
    return self.generic('GET', path, secure=secure, **r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 416, in generic
    return self.request(**r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 501, in request
    six.reraise(*exc_info)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-p

Hmm.  I'm getting a different output than him; the book says that the number of `errors` should be down to `1` at this point, but I'm still seeing three.  In fact, I don't see any meaningful difference in the `stderr`s before versus after implementing the empty function.

Well, anyways, then he says to just paste a couple lines from the `home_page` view into the new `view_list` view, so that it isn't quite so empty:

In [94]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/lists/the-only-list-in-the-world')

    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})

def view_list(request):
    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})


Overwriting lists/views.py


In [95]:
%%capture func_test_2
!python manage.py test

In [96]:
print(func_test_2.stderr)

.......FF
FAIL: test_can_start_a_list_for_one_user (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 66, in test_can_start_a_list_for_one_user
    self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 28, in wait_for_row_in_list_table
    raise e
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 24, in wait_for_row_in_list_table
    self.assertIn(row_text, [row.text for row in rows])
AssertionError: '2: Use peacock feathers to make a fly' not found in ['1: Buy peacock feathers']

FAIL: test_multiple_users_can_start_lists_at_different_urls (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last)

Well, we're back in accord with the text in terms of outputs, at least.  And the nature of what's being reported is a little less basic, so he suggests we would at this point really have to pay more attention to what it's saying, in order to figure out what we'd need to do next in order to get the tests to pass.

It's complaining about what happens when we try to add a second item to the list, so we can be pretty confident that the first item checks out.

He also notes that the unit tests should be passing at this point:

In [97]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).

.......
----------------------------------------------------------------------
Ran 7 tests in 0.032s

OK



Destroying test database for alias 'default'...


Harry says that a rule of thumb is that when all unit tests pass but the functional ones don't, it's often a problem with your templates.

In this case, the minimal home page HTML template doesn't specify what URL to POST to:

In [98]:
# %load lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            {% for item in items %}
                <tr><td> {{ forloop.counter }}: {{ item.text }}</td></tr>
            {% endfor %}
        </table>
    </body>
</html>


When that's the case, the POST request gets shunted by default back to the same URL that is currently loaded in the browser.  He says that you could change the view to include logic for handling POST requests to the new URL, too, but that that would necessitate writing a bunch of new tests, and that, instead, for his purposes at this point, it's sufficient to just tell your template file to explicitly direct the POST request to the homepage's URL (the base of the site's address, so just "`/`").  Specifically, you pass that to an "`action`" arg in the `form` tag:

In [100]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST", action="/">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            {% for item in items %}
                <tr><td> {{ forloop.counter }}: {{ item.text }}</td></tr>
            {% endfor %}
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [101]:
%%capture func_test_3
!python manage.py test

In [102]:
print(func_test_3.stderr)

........F
FAIL: test_multiple_users_can_start_lists_at_different_urls (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 92, in test_multiple_users_can_start_lists_at_different_urls
    self.assertNotIn('Buy peacock feathers', page_text)
AssertionError: 'Buy peacock feathers' unexpectedly found in 'Your To-Do list\n1: Buy peacock feathers'

----------------------------------------------------------------------
Ran 9 tests in 21.113s

FAILED (failures=1)



So that resolves the "*regression*" mentioned earlier: that (while the new functionality is not yet working as intended) at least our older tests are back to passing.


# Refactoring

Harry now advocates taking another look at our unit tests file at this point, and seeing if there are things we no longer need:

In [103]:
!grep -E "class|def" lists/tests.py

'grep' is not recognized as an internal or external command,
operable program or batch file.


Oops; I forgot that this is running in the Windows OS context:

In [105]:
with open('lists/tests.py', 'r') as f:
    lines = f.readlines()

for line in lines:
    if ('class' in line) or ('def' in line):
        print(line, end='')

class HomePageTest(TestCase):
    def test_home_page_returns_correct_html(self):
    def test_can_save_a_POST_request(self):
    def test_redirects_after_POST(self):
    def test_only_saves_items_when_necessary(self):
    def test_displays_all_list_items(self):
class ItemModelTest(TestCase):
    def test_saving_and_retrieving_items(self):
class ListViewTest(TestCase):
    def test_displays_all_items(self):


He suggests that, of these, the `test_displays_all_list_items` method is no longer needed:

In [108]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(
            response['location'], '/lists/the-only-list-in-the-world'
        )

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


class ListViewTest(TestCase):

    def test_displays_all_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/lists/the-only-list-in-the-world/')

        self.assertContains(response, 'itemey 1')
        self.assertContains(response, 'itemey 2')


Overwriting lists/tests.py


In [109]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


......
----------------------------------------------------------------------
Ran 6 tests in 0.036s

OK


So we note that the number of unit tests reported as having been performed is reduced to 6.

The next thing we recommend changing is to split out the functionality of the home page to be between prompting the user for new items to add, and another to simply display the current contents of a user's to-do list.  That involves creating a new template and, I suppose after that, either a new view or at least modifying the existing ones?

First though (of course), we write the unit test to look for the new page that just shows contents.

In [111]:
%%writefile lists/tests.py
from django.test import TestCase
from lists.models import Item

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        self.client.post('/', data={'item_text': 'A new list item'})

        self.assertEqual(Item.objects.count(), 1)
        new_item = Item.objects.first()
        self.assertEqual(new_item.text, 'A new list item')

    def test_redirects_after_POST(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertEqual(response.status_code, 302)
        self.assertEqual(
            response['location'], '/lists/the-only-list-in-the-world'
        )

    def test_only_saves_items_when_necessary(self):
        self.client.get('/')
        self.assertEqual(Item.objects.count(), 0)


class ItemModelTest(TestCase):
    
    def test_saving_and_retrieving_items(self):
        first_item = Item()
        first_item.text = 'The first (ever) list item'
        first_item.save()

        second_item = Item()
        second_item.text = 'Item the second'
        second_item.save()

        saved_items = Item.objects.all()
        self.assertEqual(saved_items.count(), 2)

        first_saved_item = saved_items[0]
        second_saved_item = saved_items[1]
        self.assertEqual(first_saved_item.text, 'The first (ever) list item')
        self.assertEqual(second_saved_item.text, 'Item the second')


class ListViewTest(TestCase):

    def test_uses_list_template(self):
        response = self.client.get('/lists/the-only-list-in-the-world/')
        self.assertTemplateUsed(response, 'list.html')

    def test_displays_all_items(self):
        Item.objects.create(text='itemey 1')
        Item.objects.create(text='itemey 2')

        response = self.client.get('/lists/the-only-list-in-the-world/')

        self.assertContains(response, 'itemey 1')
        self.assertContains(response, 'itemey 2')


Overwriting lists/tests.py


Then, just confirm that the new unit test fails:

In [112]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


......F
FAIL: test_uses_list_template (lists.tests.ListViewTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 53, in test_uses_list_template
    self.assertTemplateUsed(response, 'list.html')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\testcases.py", line 583, in assertTemplateUsed
    % (template_name, ', '.join(template_names))
AssertionError: False is not true : Template 'list.html' was not a template used to render the response. Actual template(s) used: home.html

----------------------------------------------------------------------
Ran 7 tests in 0.041s

FAILED (failures=1)


Then, we change the view to serve up the new page:

In [117]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/lists/the-only-list-in-the-world')

    items = Item.objects.all()
    return render(request, 'home.html', {'items': items})

def view_list(request):
    items = Item.objects.all()
    return render(request, 'list.html', {'items': items})


Overwriting lists/views.py


Which now points to a nonexisting template for the page:

In [119]:
%%capture unit_test_1
!python manage.py test lists

In [120]:
print(unit_test_1.stderr[-200:])

template_name, chain=chain)
django.template.exceptions.TemplateDoesNotExist: list.html

----------------------------------------------------------------------
Ran 7 tests in 0.077s

FAILED (errors=2)



So, he advocates just making an empty template file to correspond to what is sought according to the URL:

In [121]:
with open('lists/templates/list.html', 'w') as f:
    pass

In [122]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.....F.
FAIL: test_displays_all_items (lists.tests.ListViewTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 61, in test_displays_all_items
    self.assertContains(response, 'itemey 1')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\testcases.py", line 393, in assertContains
    self.assertTrue(real_count != 0, msg_prefix + "Couldn't find %s in response" % text_repr)
AssertionError: False is not true : Couldn't find 'itemey 1' in response

----------------------------------------------------------------------
Ran 7 tests in 0.042s

FAILED (failures=1)


Which also matches up with the text at this point; so it's loading the thing, and it doesn't find the specific items in the list.

Harry says that the needs at this point of what the old homepage template and the read-only list view page are similar enough that you can get started by just copying the `home.html` template file in place of the blank one we just made:

In [123]:
import shutil

shutil.copy('lists/templates/home.html', 'lists/templates/list.html')

'lists/templates/list.html'

Check that that gets your unit test back to passing (i.e., find "`itemey_1`"):

In [124]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.......
----------------------------------------------------------------------
Ran 7 tests in 0.033s

OK


Cool.  Now, modify the contents of those templates; as mentioned, we're now envisioning the home page to just prompt you to... oh, I see.  The breakdown of intended functionality for the two pages is different than I stated above.  Rather than having the homepage load your list and prompt you to insert new items *without* showing the existing items, the default assumption is that it should prompt you to create an entirely new list.

That means we can remove the lines ssaying to display the table, and further that we can change the header tag to prompt you to create a new list:

In [126]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Start a new To-Do list</h1>
        <form method="POST", action="/">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
    </body>
</html>


Overwriting lists/templates/home.html


Real quick re-run your unit tests to ensure that didn't break anything:

In [127]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.......
----------------------------------------------------------------------
Ran 7 tests in 0.041s

OK


Next, since we're not displaying preexisting list items in the home page, we can modify the view function that's returning it to avoid reference to such content:

In [129]:
%%writefile lists/views.py
from django.shortcuts import redirect, render
from lists.models import Item

def home_page(request):
    if request.method == 'POST':
        Item.objects.create(text=request.POST['item_text'])
        return redirect('/lists/the-only-list-in-the-world')
    return render(request, 'home.html')

def view_list(request):
    items = Item.objects.all()
    return render(request, 'list.html', {'items': items})


Overwriting lists/views.py


In [130]:
!python manage.py test lists

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.......
----------------------------------------------------------------------
Ran 7 tests in 0.032s

OK


So now, proceed to checking on how the functional tests are faring:

In [131]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


........F
FAIL: test_multiple_users_can_start_lists_at_different_urls (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 100, in test_multiple_users_can_start_lists_at_different_urls
    self.wait_for_row_in_list_table('1: Buy milk')
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 28, in wait_for_row_in_list_table
    raise e
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 24, in wait_for_row_in_list_table
    self.assertIn(row_text, [row.text for row in rows])
AssertionError: '1: Buy milk' not found in ['1: Buy peacock feathers', '2: Buy milk']

----------------------------------------------------------------------
Ran 9 tests in 31.419s

FAILED (failures=1)


He explains this outcome as saying that the experience of Francis' visit is marred by his added list item not showing up, and the site instead showing Edith's list (well, because we haven't actually implemented the dev code changes required to accommodate multiple users to the site).  So we've basically done all of thise legwork to restructure our project to *be ready to stage those important changes*, but haven't actually done it yet.

Harry emphasizes at this point that you shouldn't take all the work in the first half of this chapter as useless or find the process too discouraging.  Our design assumption when starting the project, of delivering minimal viable product by making a site that was only ever intended for one user, is vastly different from a site serving many users.  That's a significant alteration, and you can't make it too hastily, so setting the stage to ensure your tests and project layout are ready for it is important.

Accordingly, he suggests a commit to the repo at this point.